# Main

In [4]:
import utm
import numpy as np
import requests
import json
from dotenv import load_dotenv
import os

In [3]:
# Carga las variables de entorno desde el archivo .env en el directorio actual
load_dotenv()

True

## Lectura de datos de un archivo txt

In [3]:

# Abre el archivo en modo de lectura
archivo = open('data/coordinates.txt', 'r')

# Lee el contenido del archivo
contenido = archivo.read()

# Cierra el archivo
archivo.close()

## Conversión del string leido a una colección de objetos

In [4]:
contenido_striped = contenido.strip()

contenido_splited = contenido_striped.split(' ')

contenido_double_splited = []

for i in range(len(contenido_splited)):
    contenido_double_splited.append(contenido_splited[i].split(','))

parsed_data = []

for i in range(len(contenido_double_splited)):
    parsed_data.append({
        'longitud': float(contenido_double_splited[i][0]),
        'latitud': float(contenido_double_splited[i][1])
    })

lat_long_data=parsed_data.copy()

print(np.array(parsed_data))

[{'longitud': -71.50822803103384, 'latitud': -15.52502171075457}
 {'longitud': -71.5075531275322, 'latitud': -15.52504294763689}
 {'longitud': -71.50713003240948, 'latitud': -15.52510240816901}
 {'longitud': -71.50674347994195, 'latitud': -15.52531972263328}
 {'longitud': -71.50628580239709, 'latitud': -15.52552272214018}
 {'longitud': -71.50574878376794, 'latitud': -15.52564346399782}
 {'longitud': -71.50540610829403, 'latitud': -15.52559223856239}
 {'longitud': -71.5051914121306, 'latitud': -15.52551596548126}]


## Convirtiendo las coordenadas a UTM

In [5]:
utm_coordinates=[]

for i in range(len(parsed_data)):
    [x, y, zone_number, zone_letter] = utm.from_latlon(parsed_data[i]['latitud'], parsed_data[i]['longitud'])
    utm_coordinates.append({
        'x': float(x),
        'y': float(y),
        'zone_number': zone_number,
        'zone_letter': zone_letter
    })

print(np.array(utm_coordinates))

[{'x': 230941.8276786802, 'y': 8282025.881516408, 'zone_number': 19, 'zone_letter': 'L'}
 {'x': 231014.29217347817, 'y': 8282024.379772708, 'zone_number': 19, 'zone_letter': 'L'}
 {'x': 231059.77981020417, 'y': 8282018.329677956, 'zone_number': 19, 'zone_letter': 'L'}
 {'x': 231101.5500846214, 'y': 8281994.7589541515, 'zone_number': 19, 'zone_letter': 'L'}
 {'x': 231150.93547840498, 'y': 8281972.862286303, 'zone_number': 19, 'zone_letter': 'L'}
 {'x': 231208.72968296375, 'y': 8281960.171273797, 'zone_number': 19, 'zone_letter': 'L'}
 {'x': 231245.44222787087, 'y': 8281966.272764398, 'zone_number': 19, 'zone_letter': 'L'}
 {'x': 231268.38643696957, 'y': 8281974.986131702, 'zone_number': 19, 'zone_letter': 'L'}]


## Agregando la altitud a cada punto

### Diviendo el arreglo porque no se pueden solicitar mas de 512 puntos al mismo tiempo

In [6]:
partials=[]

tamano_subarreglo=500

for i in range(0, len(lat_long_data), tamano_subarreglo):
    subarreglo = lat_long_data[i:i + tamano_subarreglo]
    partials.append(subarreglo)

print(np.array(partials))

[[{'longitud': -71.50822803103384, 'latitud': -15.52502171075457}
  {'longitud': -71.5075531275322, 'latitud': -15.52504294763689}
  {'longitud': -71.50713003240948, 'latitud': -15.52510240816901}
  {'longitud': -71.50674347994195, 'latitud': -15.52531972263328}
  {'longitud': -71.50628580239709, 'latitud': -15.52552272214018}
  {'longitud': -71.50574878376794, 'latitud': -15.52564346399782}
  {'longitud': -71.50540610829403, 'latitud': -15.52559223856239}
  {'longitud': -71.5051914121306, 'latitud': -15.52551596548126}]]


### Petición a google cloud

In [5]:
# Settings
url_root = "https://maps.googleapis.com/maps/api/elevation"
key_str = os.getenv('GOOGLE_CLOUD_API_KEY')
output_fmt = "json"
request_str = "%s/%s?locations=%s&key=%s"
input_list = [(40.714728, -73.998672), (40.714728, -73.998672)]
output_file = "elevations.json"

In [8]:
# Prepare request string

partials_for_cloud=[]

for i in range(len(partials)):
    tuples = []
    for j in range(len(partials[i])):
        tuples.append((partials[i][j]['latitud'], partials[i][j]['longitud']))

    partials_for_cloud.append(tuples)

print(partials_for_cloud)

[[(-15.52502171075457, -71.50822803103384), (-15.52504294763689, -71.5075531275322), (-15.52510240816901, -71.50713003240948), (-15.52531972263328, -71.50674347994195), (-15.52552272214018, -71.50628580239709), (-15.52564346399782, -71.50574878376794), (-15.52559223856239, -71.50540610829403), (-15.52551596548126, -71.5051914121306)]]


In [9]:
# r_partial=requests.get(request_str % (url_root, output_fmt, "|".join([",".join([str(y) for y in x]) for x in input_list]), key_str))

responses = []

for i in range(len(partials_for_cloud)):
    responses.append(requests.get(request_str % (url_root, output_fmt, "|".join([",".join([str(y) for y in x]) for x in partials_for_cloud[i]]), key_str)))

print(responses)

[<Response [200]>]


In [10]:
# Parseo de la respuesta de google cloud
responses_parsed = []

for i in range(len(responses)):
    responses_parsed.append(json.loads(responses[i].text)["results"])

print(responses_parsed)
# print(json.loads(r.text)["results"])

[[{'elevation': 3811.70751953125, 'location': {'lat': -15.52502171075457, 'lng': -71.50822803103384}, 'resolution': 152.7032318115234}, {'elevation': 3811.9892578125, 'location': {'lat': -15.52504294763689, 'lng': -71.5075531275322}, 'resolution': 152.7032318115234}, {'elevation': 3811.9072265625, 'location': {'lat': -15.52510240816901, 'lng': -71.50713003240948}, 'resolution': 152.7032318115234}, {'elevation': 3807.909912109375, 'location': {'lat': -15.52531972263328, 'lng': -71.50674347994195}, 'resolution': 152.7032318115234}, {'elevation': 3806.519775390625, 'location': {'lat': -15.52552272214018, 'lng': -71.50628580239709}, 'resolution': 152.7032318115234}, {'elevation': 3808.522216796875, 'location': {'lat': -15.52564346399782, 'lng': -71.50574878376794}, 'resolution': 152.7032318115234}, {'elevation': 3818.451416015625, 'location': {'lat': -15.52559223856239, 'lng': -71.50540610829403}, 'resolution': 152.7032318115234}, {'elevation': 3828.814453125, 'location': {'lat': -15.52551

In [11]:
print(json.loads(responses[0].text)['results'][0])

{'elevation': 3811.70751953125, 'location': {'lat': -15.52502171075457, 'lng': -71.50822803103384}, 'resolution': 152.7032318115234}


In [12]:
# Obtencion de las elevaciones

elevations_partials = []

for i in range(len(responses_parsed)):
    elevations_partials.append([x["elevation"] for x in responses_parsed[i]])

elevations=[]

for i in range(len(elevations_partials)):
    for j in range(len(elevations_partials[i])):
        elevations.append(elevations_partials[i][j])

print(np.array(elevations))

[3811.70751953 3811.98925781 3811.90722656 3807.90991211 3806.51977539
 3808.5222168  3818.45141602 3828.81445312]


## Uniendo los datos UTM, latitud-longitud y elevación

In [13]:
joined_data = []

for i in range(len(lat_long_data)):
    joined_data.append({
        'longitud': lat_long_data[i]['longitud'],
        'latitud': lat_long_data[i]['latitud'],
        'x': utm_coordinates[i]['x'],
        'y': utm_coordinates[i]['y'],
        'zone_number': utm_coordinates[i]['zone_number'],
        'zone_letter': utm_coordinates[i]['zone_letter'],
        'elevacion': elevations[i]
    })

print(joined_data)

[{'longitud': -71.50822803103384, 'latitud': -15.52502171075457, 'x': 230941.8276786802, 'y': 8282025.881516408, 'zone_number': 19, 'zone_letter': 'L', 'elevacion': 3811.70751953125}, {'longitud': -71.5075531275322, 'latitud': -15.52504294763689, 'x': 231014.29217347817, 'y': 8282024.379772708, 'zone_number': 19, 'zone_letter': 'L', 'elevacion': 3811.9892578125}, {'longitud': -71.50713003240948, 'latitud': -15.52510240816901, 'x': 231059.77981020417, 'y': 8282018.329677956, 'zone_number': 19, 'zone_letter': 'L', 'elevacion': 3811.9072265625}, {'longitud': -71.50674347994195, 'latitud': -15.52531972263328, 'x': 231101.5500846214, 'y': 8281994.7589541515, 'zone_number': 19, 'zone_letter': 'L', 'elevacion': 3807.909912109375}, {'longitud': -71.50628580239709, 'latitud': -15.52552272214018, 'x': 231150.93547840498, 'y': 8281972.862286303, 'zone_number': 19, 'zone_letter': 'L', 'elevacion': 3806.519775390625}, {'longitud': -71.50574878376794, 'latitud': -15.52564346399782, 'x': 231208.72968

## Guardando los resultados en un archivo excel:

In [14]:
# write joined data to excel file
import pandas as pd

df = pd.DataFrame(joined_data)
df.to_excel('output/joined_data.xlsx', index=False)